In [ ]:
!pip3 install google-cloud-aiplatform --upgrade --user
!pip3 install sklearn --upgrade --user --quiet

In [ ]:
import sklearn
print('The scikit-learn version is {}.'.format(sklearn.__version__))

In [ ]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn import svm

import joblib

# Load the Iris dataset
iris = datasets.load_iris()
X = iris.data
y = iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

# Train a classifier
classifier = svm.SVC()
classifier.fit(X_train, y_train)

# Export the classifier to a file
joblib.dump(classifier, 'model.joblib')

In [ ]:
!gsutil cp model.joblib gs://aiplatform-custom/model_import

In [ ]:
from google.cloud import aiplatform

aiplatform.init(project="kubeflow-on-gcp-123", location="us-central1")

model = aiplatform.Model.upload(
    display_name='sklearn_upload',
    artifact_uri="gs://aiplatform-custom/model_import/",
    serving_container_image_uri="us-docker.pkg.dev/vertex-ai/prediction/sklearn-cpu.0-24:latest",
)

model.wait()

print(model.display_name)
print(model.resource_name)

In [ ]:
import json

items = X_test.tolist()

with open('test_data.jsonl', "w") as f:
    for i in items:
        data = i
        f.write(json.dumps(data) + "\n")
    f.close()

In [ ]:
!gsutil cp test_data.jsonl gs://aiplatform-custom/batch_input

In [ ]:
batch_prediction_job = model.batch_predict(
        job_display_name="import-batch",
        gcs_source="gs://aiplatform-custom/batch_input/test_data.jsonl",
        gcs_destination_prefix="gs://aiplatform-custom/batch_output/",
        machine_type="n1-standard-8",
        #accelerator_count=accelerator_count,
        #accelerator_type=accelerator_type,
        starting_replica_count=1,
        max_replica_count=10,
        sync=True,
    )

batch_prediction_job.wait()

print(batch_prediction_job.display_name)
print(batch_prediction_job.resource_name)
print(batch_prediction_job.state)